# Static Load Calculations for Snow Slabs

This notebook demonstrates methods for calculating the static load (weight per unit area) of layered snow slabs and their force components on slopes using the `snowpyt_mechparams` package.

## Overview

The static load calculations include:
- **Gravitational load**: Total weight per unit area (vertical force)
- **Shear load**: Force component parallel to the slope surface
- **Normal load**: Force component perpendicular to the slope surface


In [1]:
# Import Libraries
import os
import sys

import pandas as pd

# Import snowpylot for CAAML parsing
from snowpylot import caaml_parser

# Add the src directory to the path to import snowpyt_mechparams
sys.path.append('../src')
from snowpilot_utils import convert_grain_form
from snowpyt_mechparams import density
from snowpyt_mechparams.data_structures import Layer, Slab
from snowpyt_mechparams.static_load import calculate_static_load


## Parse Snowpit Files

Parse CAAML snowpit files to extract real layer data:


In [2]:
all_pits = []
failed_files = []

folder_path = 'data'

xml_files = [f for f in os.listdir(folder_path) if f.endswith('.xml')]

for file in xml_files:
    try:
        file_path = os.path.join(folder_path, file)
        pit = caaml_parser(file_path)
        all_pits.append(pit)
    except Exception as e:
        failed_files.append((file, str(e)))
        print(f"Warning: Failed to parse {file}: {e}")

print(f"Successfully parsed {len(all_pits)} files")
print(f"Failed to parse {len(failed_files)} files")


Successfully parsed 50278 files
Failed to parse 0 files


## Construct Slab

In [3]:
# C slab if ALL layers above layer of concern have successful density calculations
# Returns dictionary with pit_id -> slab mapping

slabs = {}  # Changed to dictionary to store pit_id -> slab mapping

# Track elimination steps for detailed data funnel analysis
total_pits = len(all_pits)
pits_w_slope_angle = 0
pits_w_layer_of_concern = 0
pits_w_angle_and_layer_of_concern = 0
pits_w_missing_layer_data = 0
pits_w_density_calc_failures = 0
density_calc_failures = 0  # Count individual layer failures
valid_slabs = 0

for pit in all_pits:
    pit_id = pit.core_info.pit_id
    layers = pit.snow_profile.layers

    # Track pits with individual characteristics  
    has_slope_angle = pit.core_info.location.slope_angle is not None
    has_layer_of_concern = any(getattr(layer, 'layer_of_concern', False) for layer in layers)

    if has_slope_angle:
        pits_w_slope_angle += 1
    if has_layer_of_concern:
        pits_w_layer_of_concern += 1

    # Only proceed if pit has BOTH slope angle and layer of concern
    if has_slope_angle and has_layer_of_concern:
        pits_w_angle_and_layer_of_concern += 1
        slab_layers = []
        all_layers_valid = True  # Track if all layers are successfully processed

        for layer in layers:
            if layer.layer_of_concern:
                break # Stop at layer of concern

            layer_hardness = layer.hardness
            layer_geldsetzer_grain_form = convert_grain_form(layer.grain_form_primary)

            # Reject entire pit if any layer has missing required data
            if layer_hardness is None or layer_geldsetzer_grain_form is None:
                all_layers_valid = False
                break # Stop processing this pit

            try:
                layer_density = density.calculate_density(
                    method='geldsetzer',
                    hand_hardness=layer_hardness,
                    grain_form=layer_geldsetzer_grain_form
                )
                # Handle thickness - extract value and convert from string to float
                layer_thickness = layer.thickness
                if isinstance(layer_thickness, list):
                    # First element is thickness as string, second is units
                    layer_thickness = float(layer_thickness[0]) if layer_thickness and layer_thickness[0] else 0.0
                else:
                    layer_thickness = float(layer_thickness) if layer_thickness else 0.0

                # Create layer object if density calculation succeeded
                layer_obj = Layer(thickness=layer_thickness, density=layer_density)
                slab_layers.append(layer_obj)
            except Exception as e:
                density_calc_failures += 1
                all_layers_valid = False
                break # Reject entire pit if any density calculation fails

        # Only create slab if ALL layers above layer of concern were successfully processed
        if all_layers_valid and slab_layers:
            # Handle slope angle - extract value and convert from string to float if needed
            slope_angle_raw = pit.core_info.location.slope_angle
            if isinstance(slope_angle_raw, list):
                # First element is the value (possibly as string), handle conversion
                slope_angle = float(slope_angle_raw[0]) if slope_angle_raw and slope_angle_raw[0] else 0.0
            else:
                slope_angle = float(slope_angle_raw) if slope_angle_raw else 0.0

            slab = Slab(layers=slab_layers, angle=slope_angle)
            slabs[pit_id] = slab  # Store slab with pit_id as key
            valid_slabs += 1
        else:
            # Track different types of failures
            if not slab_layers:
                pits_w_missing_layer_data += 1
            else:
                pits_w_density_calc_failures += 1


## Static Load Calculations



In [4]:
# Implement load calculation for valid slabs
import math

# Store results for analysis
static_load_results = {}

for pit_id, slab in slabs.items():
    # Convert slope angle from degrees to radians
    slope_angle_rad = math.radians(slab.angle)
    slab_thickness = slab.total_thickness

    # Calculate static load components
    gravitational_load, shear_load, normal_load = calculate_static_load(slab, slope_angle_rad)

    # Store results
    static_load_results[pit_id] = {
        'number_of_slab_layers': len(slab.layers),
        'slab_thickness': slab_thickness,
        'gravitational_load': gravitational_load,  # N/m²
        'shear_load': shear_load,                  # N/m²
        'normal_load': normal_load,                # N/m²
        'slope_angle': slab.angle,                 # degrees
        'num_layers': len(slab.layers)
    }

print(f"Calculated static loads for {len(static_load_results)} slabs")

satic_load_results_df = pd.DataFrame(static_load_results)


Calculated static loads for 10584 slabs


In [5]:
# Detailed Data Funnel - Pit Elimination Steps

print("=" * 60)
print("DATA FUNNEL: Pit Elimination Steps")
print("=" * 60)

print(f"1. Total pits processed: {total_pits:,}")

print(f"2. Pits with slope angle: {pits_w_slope_angle:,} ({(pits_w_slope_angle/total_pits*100):.1f}%)")
pits_no_slope = total_pits - pits_w_slope_angle
print(f"   └─ Eliminated (no slope angle): {pits_no_slope:,}")

print(f"3. Pits with layer of concern: {pits_w_layer_of_concern:,} ({(pits_w_layer_of_concern/total_pits*100):.1f}%)")
pits_no_layer_concern = total_pits - pits_w_layer_of_concern  
print(f"   └─ Eliminated (no layer of concern): {pits_no_layer_concern:,}")

print(f"4. Pits with BOTH slope angle AND layer of concern: {pits_w_angle_and_layer_of_concern:,} ({(pits_w_angle_and_layer_of_concern/total_pits*100):.1f}%)")
pits_missing_both = total_pits - pits_w_angle_and_layer_of_concern
print(f"   └─ Eliminated (missing slope angle OR layer of concern): {pits_missing_both:,}")

print(f"5. Pits rejected due to missing layer data: {pits_w_missing_layer_data:,}")

print(f"6. Pits rejected due to density calculation failures (Grain Form not supported by Geldsetzer): {pits_w_density_calc_failures:,}")  

print(f"7. FINAL: Valid slabs for analysis: {len(slabs):,} ({(len(slabs)/total_pits*100):.1f}%)")

print("\nSUMMARY:")
print(f"• Started with {total_pits:,} snowpit observations")
print(f"• Final dataset: {len(slabs):,} valid slabs ({(len(slabs)/total_pits*100):.1f}%)")
# Summary Stats


DATA FUNNEL: Pit Elimination Steps
1. Total pits processed: 50,278
2. Pits with slope angle: 45,515 (90.5%)
   └─ Eliminated (no slope angle): 4,763
3. Pits with layer of concern: 36,408 (72.4%)
   └─ Eliminated (no layer of concern): 13,870
4. Pits with BOTH slope angle AND layer of concern: 33,279 (66.2%)
   └─ Eliminated (missing slope angle OR layer of concern): 16,999
5. Pits rejected due to missing layer data: 12,193
6. Pits rejected due to density calculation failures (Grain Form not supported by Geldsetzer): 10,502
7. FINAL: Valid slabs for analysis: 10,584 (21.1%)

SUMMARY:
• Started with 50,278 snowpit observations
• Final dataset: 10,584 valid slabs (21.1%)


In [6]:
# Create an improved Sankey Diagram that better reflects the actual filtering logic
import plotly.graph_objects as go

# Define the data flow values (from the notebook output)
total_pits = 50278
pits_w_slope_angle = 45515
pits_w_layer_of_concern = 36408
pits_w_both = 33279  # Those with BOTH slope angle AND layer of concern
pits_rejected_missing_data = 12274
pits_rejected_density_calc = 10523
valid_slabs = 10482

# Calculate eliminations at each step
eliminated_no_slope = total_pits - pits_w_slope_angle  # 4,763
eliminated_no_layer_concern = total_pits - pits_w_layer_of_concern  # 13,870
eliminated_missing_both = total_pits - pits_w_both  # 16,999 (those eliminated for not having both criteria)

# Create a more accurate Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node = dict(
        pad = 20,
        thickness = 25,
        line = dict(color = "black", width = 0.8),
        label = [
            "Total Snowpit<br>Observations<br><b>50,278</b>",                    # 0
            "Has Slope Angle<br><b>45,515</b><br>(90.5%)",                      # 1
            "Has Layer of Concern<br><b>36,408</b><br>(72.4%)",                 # 2
            "Both Criteria Met<br><b>33,279</b><br>(66.2%)",                    # 3
            "Processing Layers<br><b>33,279</b>",                               # 4
            "Valid Slabs<br><b>10,482</b><br>(20.8%)",                         # 5
            "❌ No Slope Angle<br><b>4,763</b><br>(9.5%)",                     # 6
            "❌ No Layer of Concern<br><b>13,870</b><br>(27.6%)",              # 7
            "❌ Missing Both<br><b>16,999</b><br>(33.8%)",                     # 8
            "❌ Missing Layer Data<br><b>12,274</b><br>(24.4%)",               # 9
            "❌ Density Calc Failed<br><b>10,523</b><br>(20.9%)"               # 10
        ],
        x = [0.1, 0.3, 0.3, 0.5, 0.7, 0.9, 0.5, 0.5, 0.5, 0.9, 0.9],
        y = [0.5, 0.7, 0.3, 0.5, 0.5, 0.5, 0.9, 0.1, 0.05, 0.7, 0.3],
        color = [
            "#1f77b4",  # Total Pits - blue
            "#ff7f0e",  # With Slope Angle - orange  
            "#2ca02c",  # With Layer of Concern - green
            "#9467bd",  # Both Criteria Met - purple
            "#9467bd",  # Processing - purple
            "#2ca02c",  # Valid Slabs - green
            "#d62728",  # Eliminated - red
            "#d62728",  # Eliminated - red
            "#d62728",  # Eliminated - red
            "#d62728",  # Eliminated - red
            "#d62728"   # Eliminated - red
        ]
    ),
    link = dict(
        source = [0, 0, 0, 1, 2, 3, 4, 4, 4],  # From nodes
        target = [1, 2, 8, 6, 7, 4, 5, 9, 10], # To nodes  
        value = [
            pits_w_slope_angle,                    # Total -> Has Slope Angle
            pits_w_layer_of_concern,              # Total -> Has Layer of Concern
            eliminated_missing_both,               # Total -> Missing Both (eliminated)
            eliminated_no_slope,                   # Has Slope Angle -> No Slope Angle (eliminated)
            eliminated_no_layer_concern,           # Has Layer of Concern -> No Layer of Concern (eliminated)
            pits_w_both,                          # Both Criteria -> Processing Layers
            valid_slabs,                          # Processing -> Valid Slabs
            pits_rejected_missing_data,           # Processing -> Missing Layer Data (eliminated)
            pits_rejected_density_calc            # Processing -> Density Calc Failed (eliminated)
        ],
        color = [
            "rgba(255,127,14,0.5)",   # orange with transparency
            "rgba(44,160,44,0.5)",    # green with transparency  
            "rgba(214,39,40,0.3)",    # red with transparency (eliminated)
            "rgba(214,39,40,0.3)",    # red with transparency (eliminated)
            "rgba(214,39,40,0.3)",    # red with transparency (eliminated)
            "rgba(148,103,189,0.5)",  # purple with transparency
            "rgba(44,160,44,0.6)",    # green with transparency (success)
            "rgba(214,39,40,0.4)",    # red with transparency (eliminated)
            "rgba(214,39,40,0.4)"     # red with transparency (eliminated)
        ]
    ))])

# Update layout
fig.update_layout(
    title_text="Snow Pit Data Processing Funnel: Quality Control & Filtering Pipeline<br><sub>From 50,278 raw observations to 10,482 valid mechanical parameter datasets</sub>",
    title_x=0.5,
    font_size=11,
    height=700,
    width=1200,
    margin=dict(t=120, b=80, l=100, r=100),
    paper_bgcolor='white',
    plot_bgcolor='white'
)

# Add annotation
fig.add_annotation(
    text="<b>Data Retention Rate: 20.8%</b><br>79.2% eliminated through quality control",
    xref="paper", yref="paper",
    x=0.02, y=0.02,
    showarrow=False,
    font=dict(size=12, color="black"),
    bgcolor="lightyellow",
    bordercolor="orange",
    borderwidth=1
)

# Display the figure
fig.show()

# Create summary table
import pandas as pd
summary_data = {
    'Stage': [
        'Initial Dataset',
        'Has Slope Angle', 
        'Has Layer of Concern',
        'Both Criteria Met',
        'After Layer Processing',
        'Final Valid Slabs'
    ],
    'Count': [
        total_pits,
        pits_w_slope_angle,
        pits_w_layer_of_concern,
        pits_w_both,
        pits_w_both - pits_rejected_missing_data,
        valid_slabs
    ],
    'Percentage': [
        100.0,
        round(pits_w_slope_angle/total_pits*100, 1),
        round(pits_w_layer_of_concern/total_pits*100, 1),
        round(pits_w_both/total_pits*100, 1),
        round((pits_w_both - pits_rejected_missing_data)/total_pits*100, 1),
        round(valid_slabs/total_pits*100, 1)
    ],
    'Cumulative_Loss': [
        0,
        eliminated_no_slope,
        eliminated_no_layer_concern,
        eliminated_missing_both,
        eliminated_missing_both + pits_rejected_missing_data,
        total_pits - valid_slabs
    ]
}

summary_df = pd.DataFrame(summary_data)
print("\n" + "="*80)
print("DATA PROCESSING PIPELINE SUMMARY")
print("="*80)
print(summary_df.to_string(index=False))
print(f"\nFinal Success Rate: {valid_slabs:,} / {total_pits:,} = {(valid_slabs/total_pits*100):.1f}%")



DATA PROCESSING PIPELINE SUMMARY
                 Stage  Count  Percentage  Cumulative_Loss
       Initial Dataset  50278       100.0                0
       Has Slope Angle  45515        90.5             4763
  Has Layer of Concern  36408        72.4            13870
     Both Criteria Met  33279        66.2            16999
After Layer Processing  21005        41.8            29273
     Final Valid Slabs  10482        20.8            39796

Final Success Rate: 10,482 / 50,278 = 20.8%
